# Data Preparation

In [1]:
import os
import numpy as np
import pickle
import random
import pdb
from collections import Counter
'''
Note:  This code is just a hint for people who are not familiar with text processing in python. There is no obligation to use this code, though you may if you like. 
'''


def folder_list(path,label):
    '''
    PARAMETER PATH IS THE PATH OF YOUR LOCAL FOLDER
    '''
    filelist = os.listdir(path)
    review = []
    for infile in filelist:
        file = os.path.join(path,infile)
        r = read_data(file)
        r.append(label)
        review.append(r)
    return review

def read_data(file):
    '''
    Read each file into a list of strings. 
    Example:
    ["it's", 'a', 'curious', 'thing', "i've", 'found', 'that', 'when', 'willis', 'is', 'not', 'called', 'on', 
    ...'to', 'carry', 'the', 'whole', 'movie', "he's", 'much', 'better', 'and', 'so', 'is', 'the', 'movie']
    '''
    f = open(file)
    lines = f.read().split(' ')
    symbols = '${}()[].,:;+-*/&|<>=~" '
    words = list(map(lambda Element: Element.translate(str.maketrans({char: None for char in symbols})).strip(), lines))
    words = list(filter(lambda x:x!='', words))
    return words
	
###############################################
######## YOUR CODE STARTS FROM HERE. ##########
###############################################

def shuffle_data():
    '''
    pos_path is where you save positive review data.
    neg_path is where you save negative review data.
    '''
    pos_path = 'C:\\Users\\kumar\\Downloads\\MLCS\\Assignments\\A3\\hw3\\hw3-sentiment\\data\\data\\neg'
    neg_path = 'C:\\Users\\kumar\\Downloads\\MLCS\\Assignments\\A3\\hw3\\hw3-sentiment\\data\\data\\pos'
	
    pos_review = folder_list(pos_path,1)
    neg_review = folder_list(neg_path,-1)
	
    review = pos_review + neg_review
    random.shuffle(review)
    return review
	
    
def bag_of_words(reviews):
    '''
    Accepts a review (a list of words) and convert it into a bag of words
    Return a list of tuples containing bag of words and correxponding result
    '''
    return [(Counter(review[:-1]), review[-1]) for review in reviews]

'''
Now you have read all the files into list 'review' and it has been shuffled.
Save your shuffled result by pickle.
*Pickle is a useful module to serialize a python object structure. 
*Check it out. https://wiki.python.org/moin/UsingPickle
'''
print("Pickling the file")
pickle.dump(shuffle_data(), open("review.p","wb")) 
reviews = pickle.load( open( "review.p", "rb" ) )
train_data = reviews[:1500]
validation_data = reviews[1500:2000]
train_data = bag_of_words(train_data)
validation_dataset = bag_of_words(validation_data)
#print(train_data[1])
# Split into 1500 training and 500 validation examples



Pickling the file


# Utilities 

In [2]:
def dotProduct(d1, d2):
    """
    @param dict d1: a feature vector represented by a mapping from a feature (string) to a weight (float).
    @param dict d2: same as d1
    @return float: the dot product between d1 and d2
    """
    if len(d1) < len(d2):
        return dotProduct(d2, d1)
    else:
        return sum(d1.get(f, 0) * v for f, v in d2.items())

def increment(d1, scale, d2):
    """
    Implements d1 += scale * d2 for sparse vectors.
    @param dict d1: the feature vector which is mutated.
    @param float scale
    @param dict d2: a feature vector.

    NOTE: This function does not return anything, but rather
    increments d1 in place. We do this because it is much faster to
    change elements of d1 in place than to build a new dictionary and
    return it.
    """
    for f, v in d2.items():
        d1[f] = d1.get(f, 0) + v * scale

# Support Vector Machine via Pegasos

In [4]:
# 6.2 Normal Pagasos Implementation
from timeit import default_timer
def pegasos(Lambda, data, max_iters):
    '''
    Args
        Lambda - Regularization parameter
    Returns:
        w - a asparse weight vector w
    '''
    iters = 0
    t=1
    w=Counter()
    # a simple termination condition for now
    while(iters<max_iters):
        print("Running ", iters)
        iters = iters+1
        for data in train_data:
            t=t+1
            eta = 1.0/(t*Lambda)
            y = data[1]
            x = data[0]
            if (y*dotProduct(w,x)<1):
                increment(w, -1.0*(Lambda*eta), w)
                increment(w, eta*y ,x)
            else:
                increment(w, -1.0*(Lambda*eta), w)
    return w
print("Time for Normal Pegasos")
start = default_timer()
w_opt1 = pegasos(0.1, train_data, 1)
print(default_timer()-start)
i =0
for k,v in w_opt1.most_common():
    print(k,":", v)
    i=i+1
    if (i>10): break;

Time for Normal Pegasos
Running  0
15.494833277795522
bad : 1.1459027315123254
movie : 0.9793471019320459
only : 0.7994670219853444
have : 0.6928714190539653
? : 0.6862091938707519
no : 0.6795469686875413
at : 0.6195869420386428
they : 0.6062624916722185
even : 0.592938041305796
could : 0.5263157894736848
to : 0.4930046635576274


In [5]:
# 6.3 Optimized pagasos implementation
def pegasos_optimized(Lambda, data, max_iters, print_kinks = False):
    '''
    Args
        Lambda - Regularization parameter
    Returns:
        w - a asparse weight vector w
    '''
    iters = 0
    t=1
    w=Counter()
    s=1
    kinks=0
    # a simple termination condition for now
    while(iters<max_iters):
        iters = iters+1
        for data in train_data:
            t=t+1    
            eta = 1.0/(t*Lambda)
            y = data[1]
            x = data[0]
            # if s==0 reset the parameter
            if (s==0):
                s=1
                w=Counter()
            if print_kinks and dotProduct(x,w)==0:
                print('Feature Vector:')
                print(x)
                print("Parameter Vector:")
                print(w)
                kinks=kinks+1 
            if(y*dotProduct(x,w)<1/s):
                s = (1-eta*Lambda)*s
                increment(w, ((1/s)*eta*y) ,x)
            else: s = (1-eta*Lambda)*s
    if print_kinks: print("Number of kinks in the plot {0}".format(kinks))
    k = Counter()
    increment(k, s, w)
    return k
print("Time for optimized pegasos")
start = default_timer()
w_opt2 = pegasos_optimized(0.1, train_data, 1)
print(default_timer()-start)
i =0
for k,v in w_opt2.most_common():
    print(k,":", v)
    i=i+1
    if (i>10): break;

Time for optimized pegasos
0.4425759754991532
bad : 1.1459027315123274
movie : 0.9793471019320472
only : 0.7994670219853439
have : 0.6928714190539643
? : 0.6862091938707535
no : 0.6795469686875426
at : 0.6195869420386413
they : 0.6062624916722186
even : 0.5929380413057965
could : 0.5263157894736851
to : 0.4930046635576284


In [6]:
# 6.5 0-1 loss computation
def compute_loss(w, data):
    '''
    Return:
    percent error using 0-1 loss
    '''
    loss = 0
    for data_point in data:
        x = data_point[0]
        y = data_point[1]
        if np.sign(dotProduct(x,w)) != np.sign(y): loss=loss+1
    return (loss/len(data))*100

#compute loss on validation dataset
print("Loss with normal pegasos")
loss1 = compute_loss(w_opt1,validation_dataset)
loss2 = compute_loss(w_opt2, validation_dataset)
print(loss1)
print("Loss with optimized pegasos")
print(loss2)


Loss with normal pegasos
34.0
Loss with optimized pegasos
34.0


In [7]:
# 6.6 Search for optimal Lambda
# Write a module for searching best Lambda
def get_opt_lambda(data, lambda_range):
    '''
    Search for an optimal lambda.
    '''
    loss_hist = list()
    loss_min = np.inf
    lambda_opt = None
    # for a range of lambda calculate the loss and keep track of the minimum
    for Lambda in lambda_range:
        w = pegasos_optimized(Lambda, data, 30)
        loss = compute_loss(w, validation_dataset)
        loss_hist.append(loss)
        print("Lambda is {0}, loss is {1}".format(Lambda,loss))
        # update the minimum
        if loss<loss_min:
            lambda_opt = Lambda
            loss_min = loss
            w_opt = w
    return lambda_opt, loss_hist, w_opt

# snippet to test lambda search
print("Running")
Lambdas=list(map(lambda x:10**x,np.linspace(-6,3,10)))
lambda_opt, loss_hist, w_opt = get_opt_lambda(train_data, Lambdas)
print(lambda_opt)

Running
Lambda is 1e-06, loss is 19.0
Lambda is 1e-05, loss is 19.0
Lambda is 0.0001, loss is 20.4
Lambda is 0.001, loss is 17.2
Lambda is 0.01, loss is 18.0
Lambda is 0.1, loss is 17.2
Lambda is 1.0, loss is 20.599999999999998
Lambda is 10.0, loss is 35.8
Lambda is 100.0, loss is 50.2
Lambda is 1000.0, loss is 50.0
0.001


In [ ]:
# Loss against lambda
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(Lambdas,loss_hist)
plt.xlabel('Lambda')
plt.ylabel('Loss')
plt.xscale('log')
plt.title('Loss vs. Lambda')
plt.show()

In [ ]:
# Zoomed in Lambda search
Lambdas=list(map(lambda x:10**x,np.linspace(-4,0,10)))
lambda_opt, loss_hist,w_opt = get_opt_lambda(train_data, Lambdas)
print(lambda_opt)

In [ ]:
plt.plot(Lambdas,loss_hist)
plt.xlabel('Lambda')
plt.ylabel('Loss')
plt.xscale('log')
plt.title('Loss vs. Lambda')
plt.show()

In [ ]:
# 6.7 Plotting score against pecent error 
import pandas as pd
validation_scores = list()
error = list()
for data in validation_dataset:
    x= data[0]
    y= data[1]
    score = dotProduct(x,w_opt)
    validation_scores.append(score)
    error.append(np.sign(y)!=np.sign(score))
data = pd.DataFrame(validation_scores)
data['error'] = error
data.columns = ['score', 'error']
groups = data.groupby(pd.cut(data.score,10))
groups.sum()

As is clear from above, the error is highest in the range with smallest absolute values of scores. Hence, higher the magnitude of score, higher are the chances of the prediction being correct; We can therefore think of magnitude of score as the confidence of our prediction.

In [ ]:
#6.8 Investigating the kinks
# On Training Data
w_temp = pegasos_optimized(lambda_opt, train_data, 31, print_kinks=True)

# On validation dataset
validation_kinks = 0
for data in validation_dataset:
    x= data[0]
    y= data[1]
    if (dotProduct(x,w_temp)==0): validation_kinks+=1
print(validation_kinks)

Findings:
1. There was only one point during the gradient descent when the dot product was zero
2. That was in begining when the feature vector was empty.
3. At any other point in the algorithm its highly improbable to have a zero dot product.
4. Hence, we can skip update when the product goes to zero only if we can find a different way to initialize the parameter vector(other then empty initialization, may be randomly), not otherwise.

# Error analysis

In [ ]:
# List of wrongly predicted reviews
wrong_predictions = list()
for data in validation_dataset:
    x=data[0]
    y=data[1]
    if (y*dotProduct(x,w_opt)<0):
        wrong_predictions.append((x,y,dotProduct(x,w_opt)))
i=0
for review in wrong_predictions[:5]:
    features = review[0]  # a dict
    target = review[1]
    score = review[2]
    # get sorted w_i*x_ifor this review
    contributions = Counter()
    print("Wrong prediction {0} has true_value {1} and score of {2}".format(i,target,score))
    print("Following are the 10 greatest contributors to the predicted value")
    for word,count in features.most_common():
        contributions[word] = abs(count*w_opt[word])
    for word,contribution in contributions.most_common(10):
        print(word, "   ", contribution, "   ", features[word], "   ", w_opt[word])
    i=i+1

# Features

In [ ]:
import nltk

In [ ]:
# Download stopwords from nltk

In [ ]:
from nltk.corpus import stopwords

In [ ]:
s_words = set(stopwords.words('english'))
def remove_s_words(bw):
    '''
    Accepts a bag of words representation of input
    Returns a bag of words with stop words removed.
    '''
    s_words = set(stopwords.words('english'))
    for key,value in bw.most_common():
        if key in s_words:
            del bw[key]
    return bw
    

In [ ]:
# Generate a filtered dataset and use it for training model and test for improvement in accuracy
def new_pegasos_optimized(Lambda, data, max_iters, remove_stopwords=False):
    '''
    Args
        Lambda - Regularization parameter
    Returns:
        w - a asparse weight vector w
    '''
    iters = 0
    t=1
    w=Counter()
    s=1
    # a simple termination condition for now
    while(iters<max_iters):
        iters = iters+1
        for data in train_data:
            t=t+1    
            eta = 1.0/(t*Lambda)
            y = data[1]
            x = data[0]
            if remove_stopwords:
                x = remove_s_words(x)
            # if s==0 reset the parameter
            if (s==0):
                s=1
                w=Counter()
            if(y*dotProduct(x,w)<1/s):
                s = (1-eta*Lambda)*s
                increment(w, ((1/s)*eta*y) ,x)
            else: s = (1-eta*Lambda)*s
    k = Counter()
    increment(k, s, w)
    return k

In [ ]:
# Get optimal predictor
w_opt_5 = new_pegasos_optimized(lambda_opt, train_data, 31)
print("Loss with no optimization")
print(compute_loss(w_opt_5, validation_dataset))
w_opt_6 = new_pegasos_optimized(lambda_opt, train_data, 31, remove_stopwords=True)
print("Loss with stop words removed")
print(compute_loss(w_opt_6, validation_dataset))

## Adding Features to improve performance

In [ ]:
# Adding a n-grams to our featureset
def add_ngram(review):
    '''
    Accpets a review and adds all consequtive words as a feature
    '''
    prev = ' '
    ngrams = list()
    for word in review[:-1]:
        ngram = prev + " " + word
        ngrams.append(ngram)
        pre = word
    return ngrams + review 

def not_features(review):
    '''
    Accpets a review and combines not and following word to form a 
    feature that is added to original list of features
    '''
    prev = ' '
    ngrams = list()
    for word in review[:-1]:
        if prev == 'not':
            ngram = prev + " " + word
            ngrams.append(ngram)
            pre = word
    return ngrams + review


In [ ]:
# Create updated datasets
reviews
updated_reviews = list(map(lambda x:add_ngram(x), reviews))
updated_reviews2 = list(map(lambda x:not_features(x), updated_reviews))
# split into train and test set
updated_train = bag_of_words(updated_reviews2[:1500])
updated_validation = bag_of_words(updated_reviews2[1500:2000])

# 
w_opt_7 = new_pegasos_optimized(lambda_opt, updated_train, 31, remove_stopwords=True)
print(len(w_opt_7))
print(compute_loss(w_opt_7, updated_validation))

There is slight improvement with additione of new features